In [15]:
# Check the cluster
from dask.distributed import Client
import joblib
client = Client("3.15.27.177:8786")
client

Client Scheduler: tcp://3.15.27.177:8786 Dashboard: http://3.15.27.177:8787/status,Cluster Workers: 12 Cores: 12 Memory: 50.31 GB


In [1]:
import pandas as pd
from os import listdir
import re, joblib, gzip, gc, warnings
from pandas.io.json import json_normalize
from joblib import Parallel, delayed
warnings.filterwarnings('ignore')

In [2]:
msg_root='./data/channel_msg/'
# list all 2019 files
daily_folders=[s+'/' for s in listdir(msg_root) if '2019' in s]
# find numbers 
channel_done=[re.findall(r'\d+', s)[0] for s in listdir(msg_root+'channel/') if 'df_' in s]

In [3]:
daily_folders, channel_done


(['2019-12-23/'],
 ['1050512740',
  '1119394801',
  '1077688074',
  '1066791689',
  '1038976893',
  '1107212914',
  '1122313490',
  '1121301966',
  '1104028405',
  '1022574258',
  '1119983390',
  '1051283327',
  '1099491088',
  '1138151253',
  '1088756981',
  '1035312283',
  '1123257091',
  '1005640892',
  '1056767742',
  '1128365706',
  '1069452092',
  '1106240275',
  '1057176757',
  '1055638806',
  '1127055716',
  '1112061004',
  '1080471152',
  '1053161069',
  '1105141678',
  '1065851735',
  '1117165753',
  '1010154692',
  '1078706642',
  '1140394567',
  '1058143795',
  '1024344312',
  '1033881722',
  '1006503122',
  '1008993262',
  '1076595214',
  '1102400705',
  '1129761450',
  '1003099698',
  '1127403935',
  '1069232420',
  '1086892445',
  '1070754739',
  '1078655794',
  '1122427152',
  '1055756916',
  '1002316887',
  '1094733121',
  '1070664615',
  '1050025710',
  '1120157672',
  '1089566100',
  '1120348901',
  '1002549847',
  '1109008823',
  '1077529324',
  '1086447202',
  '109

In [30]:
# Check error files
def func_error_file_list(daily_folder):
    error_file_list=[]
    file_df_list=[s for s in listdir(msg_root+daily_folder) if 'py' not in s]
    for file_name in file_df_list:
        try:
            joblib.load(gzip.open(msg_root+daily_folder+file_name))
        except:
            error_file_list+=[daily_folder+file_name]
    gc.collect()
    return error_file_list

In [7]:
t=Parallel(n_jobs=-1)(delayed(func_error_file_list)(daily_folder) for daily_folder in daily_folders)

In [31]:
print([s for s in t if s!=[]])

[['2019-12-23/.DS_Store']]


In [32]:
# Compile data by channels
channel_set=set()
for daily_folder in daily_folders:
    [channel_set.add(s.split('_')[0]) for s in listdir(msg_root+daily_folder) if 'T' in s and s.split('_')[0] not in channel_done]
len(channel_set)
channel_set

{'1000274435',
 '1000387459',
 '1000910297',
 '1001142398',
 '1001188085',
 '1001595379',
 '1001997501',
 '1002316887',
 '1002362600',
 '1002549847',
 '1002993583',
 '1003099698',
 '1003824281',
 '1003986494',
 '1004276399',
 '1004793680',
 '1005002227',
 '1005640892',
 '1006130365',
 '1006313407',
 '1006503122',
 '1006539822',
 '1006559125',
 '1006832377',
 '1007454380',
 '1007990084',
 '1008635019',
 '1008944300',
 '1008993262',
 '1010154692',
 '1011047399',
 '1012939624',
 '1014195192',
 '1015395776',
 '1016002406',
 '1017350277',
 '1017438514',
 '1019701035',
 '1021345616',
 '1022574258',
 '1024048235',
 '1024344312',
 '1027214245',
 '1030924969',
 '1031176887',
 '1033660304',
 '1033812718',
 '1033881722',
 '1034335300',
 '1035312283',
 '1035568251',
 '1036240821',
 '1036683988',
 '1036993118',
 '1037899224',
 '1038336034',
 '1038541046',
 '1038976893',
 '1040701586',
 '1041180794',
 '1041903680',
 '1044805702',
 '1045863670',
 '1046337720',
 '1048560351',
 '1048695410',
 '10500257

In [33]:
# Drop duplicates and save memory.
def func_cleanup(dataframe):
    dataframe=dataframe.loc[dataframe[['to_id.channel_id', 'id']].drop_duplicates().index]
    gc.collect()
    return dataframe

def func_channel(channel_id):
    df_channel=pd.DataFrame()
    for daily_folder in daily_folders:
        
        def func_read_json(file_name):
            json_read=joblib.load(gzip.open(msg_root+daily_folder+file_name))
            df_json_norm_temp=json_normalize(json_read)
            return df_json_norm_temp
        
        channel_df_files=[s for s in listdir(msg_root+daily_folder) if channel_id in s]
        if channel_df_files!=[]:
            df_channel=func_cleanup(pd.concat([df_channel]+[func_read_json(s) for s in channel_df_files], ignore_index=True))
            gc.collect()
#             print(daily_folder, channel_id)
        else:
            pass
    df_channel=func_cleanup(df_channel)
    df_channel.to_pickle(msg_root+'channel/'+'df_telegram_hk_'+channel_id+'.pkl.gz', compression='gzip')
    gc.collect()

In [35]:
%time _ = Parallel(n_jobs=-1)(delayed(func_channel)(channel_id) for channel_id in channel_set)

CPU times: user 4.12 s, sys: 348 ms, total: 4.47 s
Wall time: 37min 18s


In [15]:
msg_root='./data/channel_msg/channel/'
# list all 2019 files
daily_folders=[s+'/' for s in listdir(msg_root) ]
daily_folders

['df_telegram_hk_1050512740.pkl.gz/',
 'df_telegram_hk_1119394801.pkl.gz/',
 'df_telegram_hk_1077688074.pkl.gz/',
 'df_telegram_hk_1066791689.pkl.gz/',
 'df_telegram_hk_1038976893.pkl.gz/',
 'df_telegram_hk_1107212914.pkl.gz/',
 'df_telegram_hk_1122313490.pkl.gz/',
 'df_telegram_hk_1121301966.pkl.gz/',
 'df_telegram_hk_1104028405.pkl.gz/',
 'df_telegram_hk_1022574258.pkl.gz/',
 'df_telegram_hk_1119983390.pkl.gz/',
 'df_telegram_hk_1051283327.pkl.gz/',
 'df_telegram_hk_1099491088.pkl.gz/',
 'df_telegram_hk_1138151253.pkl.gz/',
 'df_telegram_hk_1088756981.pkl.gz/',
 'df_telegram_hk_1035312283.pkl.gz/',
 'df_telegram_hk_1123257091.pkl.gz/',
 'df_telegram_hk_1005640892.pkl.gz/',
 'df_telegram_hk_1056767742.pkl.gz/',
 'df_telegram_hk_1128365706.pkl.gz/',
 'df_telegram_hk_1069452092.pkl.gz/',
 'df_telegram_hk_1106240275.pkl.gz/',
 'df_telegram_hk_1057176757.pkl.gz/',
 'df_telegram_hk_1055638806.pkl.gz/',
 'df_telegram_hk_1127055716.pkl.gz/',
 'df_telegram_hk_1112061004.pkl.gz/',
 'df_telegra

In [14]:
df_test = pd.read_pickle(msg_root+daily_folders[0][:-1], compression="gzip")
df_test.head()

,_,id,date,message,out,mentioned,media_unread,silent,post,from_scheduled,...,action._,action.photo._,action.photo.id,action.photo.access_hash,action.photo.file_reference,action.photo.date,action.photo.sizes,action.photo.dc_id,action.photo.has_stickers,action.title
0,Message,88,2019-11-25 11:43:18+00:00,盛康選區500多張問題票突然翻生\n民主派趙寶琴由贏變輸 支持者鼓譟\n\n節錄內文：\n\...,False,False,False,False,True,False,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
1,Message,87,2019-11-25 01:10:16+00:00,2019年區議會選舉結果 － 青衣\n\nS21 安灝 － 譚家浚（公民黨）\nS22 偉盈...,False,False,False,False,True,False,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2,Message,86,2019-09-13 09:36:25+00:00,[#交通] 今晚中秋節通宵交通服務\n\n今晚（9月13日晚上-9月14日凌晨）九巴青衣路線...,False,False,False,False,True,False,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
3,Message,85,2019-09-01 14:45:04+00:00,機場快綫由機場往九龍站/香港站重開（不停青衣站），機場快綫往機場方向、東涌綫全綫及迪士尼站繼...,False,False,False,False,True,False,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
4,Message,84,2019-09-01 10:34:27+00:00,機場快綫、東涌綫、迪士尼綫，全綫暫停服務,False,False,False,False,True,False,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
